## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database2.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,39.94,95,38,3.78,scattered clouds
1,1,Clyde River,CA,70.4692,-68.5914,-10.97,77,100,17.27,light snow
2,2,Bluff,NZ,-46.6000,168.3333,72.81,84,100,7.27,light rain
3,3,Hamilton,US,39.1834,-84.5333,39.72,87,100,1.01,mist
4,4,Mar Del Plata,AR,-38.0023,-57.5575,72.09,92,0,3.00,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bluff,NZ,-46.6000,168.3333,72.81,84,100,7.27,light rain
4,4,Mar Del Plata,AR,-38.0023,-57.5575,72.09,92,0,3.00,clear sky
5,5,Kendari,ID,-3.9450,122.4989,87.84,60,49,3.47,scattered clouds
7,7,Bandarbeyla,SO,9.4942,50.8122,74.41,66,0,15.97,clear sky
8,8,Itarema,BR,-2.9248,-39.9167,78.62,79,17,10.36,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             242
City                   242
Country                242
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bluff,NZ,-46.6000,168.3333,72.81,84,100,7.27,light rain
4,4,Mar Del Plata,AR,-38.0023,-57.5575,72.09,92,0,3.00,clear sky
5,5,Kendari,ID,-3.9450,122.4989,87.84,60,49,3.47,scattered clouds
7,7,Bandarbeyla,SO,9.4942,50.8122,74.41,66,0,15.97,clear sky
8,8,Itarema,BR,-2.9248,-39.9167,78.62,79,17,10.36,few clouds
...,...,...,...,...,...,...,...,...,...,...
667,667,Guane,CU,22.2025,-84.0875,70.68,72,97,5.57,overcast clouds
668,668,Tonj,SS,7.2737,28.6822,75.76,35,96,4.14,overcast clouds
672,672,Hambantota,LK,6.1241,81.1185,81.43,71,92,14.09,overcast clouds
673,673,Sao Domingos Do Maranhao,BR,-5.5758,-44.3853,71.73,98,100,3.11,overcast clouds


In [7]:
clean_df = preferred_cities_df

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City","Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bluff,NZ,72.81,light rain,-46.6000,168.3333,
4,Mar Del Plata,AR,72.09,clear sky,-38.0023,-57.5575,
5,Kendari,ID,87.84,scattered clouds,-3.9450,122.4989,
7,Bandarbeyla,SO,74.41,clear sky,9.4942,50.8122,
8,Itarema,BR,78.62,few clouds,-2.9248,-39.9167,
9,Esperance,AU,73.24,overcast clouds,-33.8667,121.9000,
16,Hermanus,ZA,72.90,clear sky,-34.4187,19.2345,
18,Bredasdorp,ZA,70.29,scattered clouds,-34.5322,20.0403,
24,Cape Town,ZA,73.11,clear sky,-33.9258,18.4232,
28,Port-Gentil,GA,80.78,scattered clouds,-0.7193,8.7815,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bluff,NZ,72.81,light rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
4,Mar Del Plata,AR,72.09,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
5,Kendari,ID,87.84,scattered clouds,-3.9450,122.4989,Hotel Wixel
7,Bandarbeyla,SO,74.41,clear sky,9.4942,50.8122,JABIR HOTEL
8,Itarema,BR,78.62,few clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
...,...,...,...,...,...,...,...
667,Guane,CU,70.68,overcast clouds,22.2025,-84.0875,base de Campismo LosPortales de Guane
668,Tonj,SS,75.76,overcast clouds,7.2737,28.6822,Lakan Hotel
672,Hambantota,LK,81.43,overcast clouds,6.1241,81.1185,Bungalow 63
673,Sao Domingos Do Maranhao,BR,71.73,overcast clouds,-5.5758,-44.3853,Pousada Babaçu


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna

<bound method DataFrame.dropna of                          City Country  Max Temp Current Description      Lat  \
2                       Bluff      NZ     72.81          light rain -46.6000   
4               Mar Del Plata      AR     72.09           clear sky -38.0023   
5                     Kendari      ID     87.84    scattered clouds  -3.9450   
7                 Bandarbeyla      SO     74.41           clear sky   9.4942   
8                     Itarema      BR     78.62          few clouds  -2.9248   
..                        ...     ...       ...                 ...      ...   
667                     Guane      CU     70.68     overcast clouds  22.2025   
668                      Tonj      SS     75.76     overcast clouds   7.2737   
672                Hambantota      LK     81.43     overcast clouds   6.1241   
673  Sao Domingos Do Maranhao      BR     71.73     overcast clouds  -5.5758   
677                    Luanda      AO     76.08     overcast clouds  -8.8368   

     

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<strong>Hotel Name</strong><br>
</dt>{Hotel Name}<br>
<strong>Country</strong><br>
</dt>{Country}<br>
<strong>City</strong><br>
</dt>{City}<br>
<strong>Max Temp & Currrent Description</strong><br>
</dt>{Max Temp} °F </dt>{Current Description}<br>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=100, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Call the figure to plot the data.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))